In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import warnings
warnings.filterwarnings(action='ignore')
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
train = pd.read_csv('/kaggle/input/titanic/train.csv')
test = pd.read_csv('/kaggle/input/titanic/test.csv')
#데이터 불러온 후,
train.info()
#데이터 특성 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [3]:
train_and_test = [train, test]
#전체적인 데이터 전처리를 위한 데이터 합체

# 데이터 전처리
특성별로 노트를 나누고 노트의 마지막엔 'Survived'특성과 교차정보 수집

In [4]:
#Age
for dataset in train_and_test:
    dataset['Sex'] = dataset['Sex'].astype(str)   #성별데이터 str타입으로 변환   
for dataset in train_and_test:
    dataset['Sex'] = dataset['Sex'].replace('female', 0)   #여자=0, 남자=1로 인코딩
    dataset['Sex'] = dataset['Sex'].replace('male', 1)

pd.crosstab(train['Survived'], train['Sex'])

Sex,0,1
Survived,,
0,81,468
1,233,109


In [5]:
#Embarked
for dataset in train_and_test:
    dataset['Embarked'] = dataset['Embarked'].fillna('S')  #탑승지 결측값을 데이터중 가장 많이 분포되어있는 'S'로 가정
    dataset['Embarked'] = dataset['Embarked'].astype(str)  #one-hot인코딩을 위하여 데이터 str타입으로 변환
pd.crosstab(train['Survived'], train['Embarked'])

Embarked,C,Q,S
Survived,,,
0,75,47,427
1,93,30,219


In [6]:
#Name->Title
for dataset in train_and_test:
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.')     #’ ([A-Za-z]+).‘는 정규표현식인데, 공백으로 시작하고, .으로 끝나는 문자열을 추출할 때 저렇게 표현을 한다.
for dataset in train_and_test:
    dataset['Title'] = dataset['Title'].replace(['Capt', 'Col', 'Countess', 'Don','Dona', 'Dr', 'Jonkheer', 'Lady','Major', 'Rev', 'Sir'], 'Other')    #별로 없는 데이터의 경우 'Other'로 묶음
    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')    #'Mile'='Miss' 같은의미이므로 통일
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')      #'Mme'='Mrs' 같은의미
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')      #'Ms'='Miss' 같은의미

pd.crosstab(train['Survived'], train['Title'])

Title,Master,Miss,Mr,Mrs,Other
Survived,,,,,
0,17,55,436,26,15
1,23,130,81,100,8


In [7]:
#Title->Master
for dataset in train_and_test:
    dataset.loc[ dataset['Title']!='Master', 'Master'] = 0    #'Master'특성을 새로 만들고 'Title'의 값이 'Master'면 1, 아니면 0 저장
    dataset.loc[ dataset['Title']=='Master', 'Master'] = 1
    dataset['Master'] = dataset['Master'].astype(int)         #0, 1로만 이루어진 특성이기 때문에 int타입으로 변환하여 인코딩

pd.crosstab(train['Survived'], train['Master'])

Master,0,1
Survived,,
0,532,17
1,319,23


In [8]:
#Pclass
for dataset in train_and_test:
    dataset['Pclass'] = dataset['Pclass'].astype(str)    #one-hot인코딩을 위하여 str타입으로 변환
pd.crosstab(train['Survived'], train['Pclass'])

Pclass,1,2,3
Survived,,,
0,80,97,372
1,136,87,119


In [9]:
#Age
for dataset in train_and_test:
    dataset.loc[(dataset['Title'] == 'Master'), 'ttAge'] = dataset['Age']    #'ttAge'라는 새로운 특성을 만들고, 'Title'의 값이 'Master'인 데이터의 'Age'값 저장
Mas_mean = round(dataset['ttAge'].mean(), 2)    #위에서 저장한 데이터의 평균 = 'Title'의 값이 'Master'인 사람들의 나이 평균값
for dataset in train_and_test:
    dataset.loc[(dataset['Title'] == 'Miss'), 'ttAge'] = dataset['Age']      #'Title'의 값이 'Miss'인 데이터의 'Age'값 저장
    dataset.loc[(dataset['Title'] != 'Miss'), 'ttAge'] = None                #'Title'의 값이 'Miss'가 아닌 경우 None으로 초기화
Mis_mean = round(dataset['ttAge'].mean(), 2)    # 'Title'의 값이 'Miss'인 사람들의 나이 평균값 저장
for dataset in train_and_test:
    dataset.loc[(dataset['Title'] == 'Mr'), 'ttAge'] = dataset['Age']       #'Title'의 값이 'Mr'인 데이터의 'Age'값 저장
    dataset.loc[(dataset['Title'] != 'Mr'), 'ttAge'] = None                 #'Title'의 값이 'Mr'가 아닌 경우 None으로 초기화
Mr_mean = round(dataset['ttAge'].mean(), 2)     # 'Title'의 값이 'Mr'인 사람들의 나이 평균값 저장
for dataset in train_and_test:
    dataset.loc[(dataset['Title'] == 'Mrs'), 'ttAge'] = dataset['Age']      #'Title'의 값이 'Mrs'인 데이터의 'Age'값 저장
    dataset.loc[(dataset['Title'] != 'Mrs'), 'ttAge'] = None                #'Title'의 값이 'Mrs'가 아닌 경우 None으로 초기화
Mrs_mean = round(dataset['ttAge'].mean(), 2)    # 'Title'의 값이 'Mrs'인 사람들의 나이 평균값 저장
for dataset in train_and_test:
    dataset.loc[(dataset['Title'] == 'Other'), 'ttAge'] = dataset['Age']    #'Title'의 값이 'Other'인 데이터의 'Age'값 저장
    dataset.loc[(dataset['Title'] != 'Other'), 'ttAge'] = None              #'Title'의 값이 'Other'가 아닌 경우 None으로 초기화
Oth_mean = round(dataset['ttAge'].mean(), 2)    # 'Title'의 값이 'Other'인 사람들의 나이 평균값 저장
print(Mas_mean, Mis_mean, Mr_mean, Mrs_mean, Oth_mean)

7.41 21.77 32.0 38.9 43.83


In [10]:
for dataset in train_and_test:
    dataset.loc[(dataset['Title']=='Master')&(dataset['Age'].isnull()==True), 'Age'] = Mas_mean
    dataset.loc[(dataset['Title']=='Miss')&(dataset['Age'].isnull()==True), 'Age'] = Mis_mean
    dataset.loc[(dataset['Title']=='Mr')&(dataset['Age'].isnull()==True), 'Age'] = Mr_mean
    dataset.loc[(dataset['Title']=='Mrs')&(dataset['Age'].isnull()==True), 'Age'] = Mrs_mean
    dataset.loc[(dataset['Title']=='Other')&(dataset['Age'].isnull()==True), 'Age'] = Oth_mean
#'Age'결측치에 각각의 위에서 구한 각'Title'에 대한 'Age' 평균치를 저장

pd.crosstab(train['Survived'], train['Age'])

Age,0.42,0.67,0.75,0.83,0.92,1.00,2.00,3.00,4.00,5.00,...,62.00,63.00,64.00,65.00,66.00,70.00,70.50,71.00,74.00,80.00
Survived,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,2,7,1,3,0,...,2,0,2,3,1,2,1,2,1,0
1,1,1,2,2,1,5,3,5,7,4,...,2,2,0,0,0,0,0,0,0,1


In [11]:
#Fare
for dataset in train_and_test:
    dataset['Fare'] = dataset['Fare'].fillna(0)    #'Fare'의 결측치는 test데이터에 단 1개만 존재하므로 0으로 저장
pd.crosstab(train['Survived'], train['Fare'])

Fare,0.0000,4.0125,5.0000,6.2375,6.4375,6.4500,6.4958,6.7500,6.8583,6.9500,...,153.4625,164.8667,211.3375,211.5000,221.7792,227.5250,247.5208,262.3750,263.0000,512.3292
Survived,,,,,,,,,,,,,,,,,,,,,
0,14,1,1,1,1,1,2,2,1,1,...,1,0,0,1,1,1,1,0,2,0
1,1,0,0,0,0,0,0,0,0,0,...,2,2,3,0,0,3,1,2,2,3


In [12]:
#Cabin
for dataset in train_and_test:
    dataset['Cabin'] = dataset['Cabin'].fillna('K')    #'Cabin'의 결측치는 나중에 처리하기 위해 우선 'K'로
    dataset['Cabin'] = dataset['Cabin'].str[:1]        #객실의 호수를 제외하고 종류만 따오기 위해서 맨 앞의 알파벳만 저장
    dataset['Cabin'] = dataset['Cabin'].astype(str)
pd.crosstab(train['Pclass'], train['Cabin'])           #'Cabin과 'Pclass'가 연관이 있다고 판단하여 결측치를 채울 때 'Pclass'를 고려

Cabin,A,B,C,D,E,F,G,K,T
Pclass,,,,,,,,,
1,15,47,59,29,25,0,0,40,1
2,0,0,0,4,4,8,0,168,0
3,0,0,0,0,3,5,4,479,0


In [13]:
pd.crosstab(train['Survived'], train['Cabin'])

Cabin,A,B,C,D,E,F,G,K,T
Survived,,,,,,,,,
0,8,12,24,8,8,5,2,481,1
1,7,35,35,25,24,8,2,206,0


In [14]:
for dataset in train_and_test:
    dataset.loc[(dataset['Pclass'] == '1') & (dataset['Cabin'] == 'K'), 'Cabin'] = 'C'     #결측치별로 'Pclass'에 가장 많이 분포되어있는 값으로 채워넣음
    dataset.loc[(dataset['Pclass'] == '2') & (dataset['Cabin'] == 'K'), 'Cabin'] = 'F'
    dataset.loc[(dataset['Pclass'] == '3') & (dataset['Cabin'] == 'K'), 'Cabin'] = 'G'
    dataset['Cabin'] = dataset['Cabin'].replace(['T'], 'C')
for dataset in train_and_test:
    dataset.loc[(dataset['Cabin']=='B'), 'Cabin'] = 1     #생존률이 더 높은 객실=1, 더 낮은 객실=0으로 인코딩
    dataset.loc[(dataset['Cabin']=='D'), 'Cabin'] = 1
    dataset.loc[(dataset['Cabin']=='E'), 'Cabin'] = 1
    dataset.loc[(dataset['Cabin']=='A'), 'Cabin'] = 0
    dataset.loc[(dataset['Cabin']=='C'), 'Cabin'] = 0
    dataset.loc[(dataset['Cabin']=='F'), 'Cabin'] = 0
    dataset.loc[(dataset['Cabin']=='G'), 'Cabin'] = 0
    dataset['Cabin'] = dataset['Cabin'].astype(int)

pd.crosstab(train['Survived'], train['Cabin'])

Cabin,0,1
Survived,,
0,521,28
1,258,84


In [15]:
#Parch, SibSp->Family
for dataset in train_and_test:
    dataset['Family'] = dataset['Parch']+dataset['SibSp']+1    #'Parch'데이터와 'SibSp'데이터를 합쳐서 자신을 포함한 동승자를 나타내는 'Family'라는 특성생성
pd.crosstab(train['Survived'], train['Family'])

Family,1,2,3,4,5,6,7,8,11
Survived,,,,,,,,,
0,374,72,43,8,12,19,8,6,7
1,163,89,59,21,3,3,4,0,0


In [16]:
for dataset in train_and_test:    #2~4인 가족이 생존률이 더 높은것을 확인
    dataset.loc[(dataset['Family'] < 2) | (dataset['Family'] > 4), 'Family'] = 0     #위의 조건을 충족하는 경우 1, 아닐경우 0으로 인코딩
    dataset.loc[(dataset['Family'] >= 2) & (dataset['Family'] <= 4), 'Family'] = 1
pd.crosstab(train['Survived'], train['Family'])

Family,0,1
Survived,,
0,426,123
1,173,169


In [17]:
features_drop = ['Name', 'Ticket', 'Parch', 'SibSp', 'Title', 'ttAge', 'Cabin', 'Age']   #학습할 때 버릴 특성 선택
train = train.drop(features_drop, axis=1)        #train데이터에서 위에서 선택한 특성 drop
test = test.drop(features_drop, axis=1)          #test데이터에서 위에서 선택한 특성 drop
train = train.drop(['PassengerId'], axis=1)      #추가로 train데이터의 승객번호는 필요없으므로 drop

print(train.head())
print(test.head())        #데이터 확인

   Survived Pclass  Sex     Fare Embarked  Master  Family
0         0      3    1   7.2500        S       0       1
1         1      1    0  71.2833        C       0       1
2         1      3    0   7.9250        S       0       0
3         1      1    0  53.1000        S       0       1
4         0      3    1   8.0500        S       0       0
   PassengerId Pclass  Sex     Fare Embarked  Master  Family
0          892      3    1   7.8292        Q       0       0
1          893      3    0   7.0000        S       0       1
2          894      2    1   9.6875        Q       0       0
3          895      3    1   8.6625        S       0       0
4          896      3    0  12.2875        S       0       1


In [18]:
train = pd.get_dummies(train)     #str타입데이터 가변수화로 one-hot인코딩 완성(데이터간의 관계성 제거)
test = pd.get_dummies(test)       

train_label = train['Survived']    #훈련 데이터의 target데이터 지정
train_data = train.drop('Survived', axis=1)    #target데이터로 지정한 특성 삭제(학습에서 제외)
test_data = test.drop("PassengerId", axis=1).copy()    #test데이터의 승객번호는 submission파일 제출시 사용하므로 복사해두고 predict땐 필요없으므로 삭제
train_data.head(10)       #전처리 완료된 데이터 확인

,Sex,Fare,Master,Family,Pclass_1,Pclass_2,Pclass_3,Embarked_C,Embarked_Q,Embarked_S
0,1,7.2500,0,1,0,0,1,0,0,1
1,0,71.2833,0,1,1,0,0,1,0,0
2,0,7.9250,0,0,0,0,1,0,0,1
3,0,53.1000,0,1,1,0,0,0,0,1
4,1,8.0500,0,0,0,0,1,0,0,1
5,1,8.4583,0,0,0,0,1,0,1,0
6,1,51.8625,0,0,1,0,0,0,0,1
7,1,21.0750,1,0,0,0,1,0,0,1
8,0,11.1333,0,1,0,0,1,0,0,1
9,0,30.0708,0,1,0,1,0,1,0,0


# 모델 학습
* 모델을 파라미터로 사용하면 교차검증을 통하여 훈련세트와 검증세트 각각의 점수를 확인할 수 있도록하는 함수 정의

In [19]:
def train_and_test(model):      
    scores = cross_validate(model, train_data, train_label, return_train_score=True, n_jobs=-1)    #교차검증 처리(K=5)
    model.fit(train_data, train_label)     #모델 학습
    train_accuracy = np.mean(scores['train_score'])    #교차검증 과정에서의 훈련세트 점수
    ver_accuracy = np.mean(scores['test_score'])       #교차검증 과정에서의 검증세트 점수
    print(model, "\n trn : ", round(train_accuracy*100, 2), "\n ver : ", round(ver_accuracy*100, 2))    #위에서 구한 훈련, 검증세트 점수 출력
    return model.predict(test_data)    #훈련한 모델에 대한 test데이터의 예측치 반환

* 각 모델별로 그리드서치를 통하여 하이퍼파라미터를 탐색
* 탐색한 하이퍼파라미터가 적용된 모델을 저장
* 하이퍼파라미터가 적용된 모델의 test데이터에 대한 predict값 저장
* 모델의 훈련점수와 하이퍼파라미터 확인

In [20]:
params = {'n_neighbors': range(1, 20, 2)}      #최근접 이웃 모델의 이웃수는 보통 홀수로 지정하기 때문에 1~20사이에 있는 홀수로 지정
knn_gs = GridSearchCV(KNeighborsClassifier() , params, cv=5, n_jobs=-1)     #최근접 이웃모델의 하이퍼파라미터 탐색
knn_gs.fit(train_data, train_label)    #최근접 이웃 모델 학습
knn_bs=knn_gs.best_estimator_          #위에서 구한 하이퍼파라미터를 적용한 모델 저장
knn_pd = knn_bs.predict(test_data)     #하이퍼파라미터가 적용된 모델의 predic값 저장
print(knn_bs.score(train_data, train_label))    #하이퍼파라미터가 적용된 모델의 훈련점수
print(knn_gs.best_params_)    #하이퍼파라미터 확인

0.8170594837261503
{'n_neighbors': 9}


In [21]:
params = {'loss' : ['log', 'hinge'], 'max_iter' : [5000], 'tol' : [None]}     #경사하강법의 경우 log방식과 svm방식 두가지 중에서 선정
sgd_gs = GridSearchCV(SGDClassifier(), params, cv=5, n_jobs=-1)  
sgd_gs.fit(train_data, train_label)    
sgd_bs=sgd_gs.best_estimator_         
sgd_pd = sgd_bs.predict(test_data)     
print(sgd_bs.score(train_data, train_label))    
print(sgd_gs.best_params_)   

0.7429854096520763
{'loss': 'log', 'max_iter': 5000, 'tol': None}


In [22]:
params = {'C': range(10, 50, 10), 'max_iter' : [5000]}     #로지스틱 회귀의 규제 강도는 50을 잘 넘기지 않으므로 10~50까지 범위지정
log_gs = GridSearchCV(LogisticRegression(), params, cv=5, n_jobs=-1)  
log_gs.fit(train_data, train_label)   
log_bs=log_gs.best_estimator_          
log_pd = log_bs.predict(test_data)    
print(log_bs.score(train_data, train_label))   
print(log_gs.best_params_)

0.8170594837261503
{'C': 10, 'max_iter': 5000}


In [23]:
params = {'max_depth' : range(2, 16, 1)}    #결정트리의 'max_depth'파라미터는 15를 잘 넘기지 않는것을 확인하여 2~16까지 범위지정
dt_gs = GridSearchCV(DecisionTreeClassifier(), params, cv=5, n_jobs=-1)
dt_gs.fit(train_data, train_label)
dt_bs=dt_gs.best_estimator_
dt_pd = dt_bs.predict(test_data)
print(dt_bs.score(train_data, train_label))
print(dt_gs.best_params_)
print(dt_bs.feature_importances_)

0.9180695847362514
{'max_depth': 14}
[0.39208365 0.33267072 0.04037748 0.08968741 0.00096816 0.00998717
 0.09849803 0.01774409 0.00582259 0.01216071]


In [24]:
params = {'max_depth' : range(2, 16, 1), 'n_estimators':[100], 'oob_score':[True]}    #랜덤포레스트의 'max_depth'파라미터는 15를 잘 넘기지 않는것을 확인하여 2~16까지 범위지정, 결정트리의 갯수는 100으로 우선 고정
rf_gs = GridSearchCV(RandomForestClassifier(), params, cv=5, n_jobs=-1)
rf_gs.fit(train_data, train_label)
rf_bs=rf_gs.best_estimator_
rf_pd = rf_bs.predict(test_data)
print(rf_bs.score(train_data, train_label))
print(rf_gs.best_params_)
print(rf_bs.feature_importances_)

0.9158249158249159
{'max_depth': 9, 'n_estimators': 100, 'oob_score': True}
[0.35599439 0.37212025 0.05384403 0.05620426 0.03351281 0.01918965
 0.07038182 0.01180986 0.00913928 0.01780365]


* 위에서 정의한 함수로 각 모델의 훈련, 검증세트 점수 확인
* 이 때 파라미터는 위에서 구한 각 모델의 하이퍼파라미터를 적용

In [25]:
#kNN
knn_pred = train_and_test(KNeighborsClassifier(n_neighbors=knn_gs.best_params_['n_neighbors']))
# Logistic Regression
log_pred = train_and_test(LogisticRegression(C=log_gs.best_params_['C'], max_iter=5000))
#SGDClassifier
sgd_pred = train_and_test(SGDClassifier(loss=sgd_gs.best_params_['loss'], max_iter=5000, tol=None))
#DecisionTreeClassifier
dt_pred = train_and_test(DecisionTreeClassifier(max_depth=dt_gs.best_params_['max_depth']))
#RandomForestClassifier
rf_pred = train_and_test(RandomForestClassifier(max_depth=rf_gs.best_params_['max_depth'], n_estimators=100, oob_score=True))

KNeighborsClassifier(n_neighbors=9) 
 trn :  80.89 
 ver :  76.44
LogisticRegression(C=10, max_iter=5000) 
 trn :  81.37 
 ver :  80.47
SGDClassifier(loss='log', max_iter=5000, tol=None) 
 trn :  73.18 
 ver :  72.28
DecisionTreeClassifier(max_depth=14) 
 trn :  92.2 
 ver :  84.07
RandomForestClassifier(max_depth=9, oob_score=True) 
 trn :  91.67 
 ver :  82.6


# 모델 결정 및 submission파일 생성
* 검증세트의 점수가 가장 높게 확인되는 결정트리와 랜덤포레스트 두가지 모델 선정
* 이 때 predict는 하이퍼파라미터가 적용된 모델의 예측값 사용

In [26]:
submission = pd.DataFrame({
    "PassengerId": test["PassengerId"],
    "Survived": dt_pd
})
#test데이터의 승객번호와 predict값을 합성하여 데이터 생성
submission.to_csv('submission.csv', index=False)    #생성된 데이터를 csv파일로 저장

In [27]:
submission = pd.DataFrame({
    "PassengerId": test["PassengerId"],
    "Survived": rf_pd
})

submission.to_csv('submission_rf.csv', index=False)